# Feed Listeners

## Overview
This notebook explores using LLMs (and other AI techniques) to listen to and analyse feeds of information, which in this case is a BBC News RSS feed. The goal is to evaluate whether AI can filter out noisy articles and identify those that are relevant to a specific organisation or department.

### The Experiment
The core hypothesis under test is that an LLM and/or a combination of AI techniques can effectively filter a news feed to identify articles that are relevant to a specific organisation or department, while ignoring irrelevant content by:
- Using an LLM to assess the relevance of each article based on its title and summary, and filtering out those that do not meet a certain relevance threshold.
- Applying embedding techniques to compare the content of the articles with a predefined set of keywords or topics related to the organisation or department, and filtering out those that do not have a strong semantic similarity.
- Analysing the sentiment of the articles to determine if they are positive, negative, or neutral towards the organisation or department in question.

### What is in scope?
The experiment will focus on the following aspects:
- Ingesting a news feed ([BBC News Front Page RSS](http://feeds.bbci.co.uk/news/rss.xml) in this case)
- Using and evaluating LLMs to filter the feed for relevance to a specific organisation or department
- Evaluating the sustainability of using LLMs for this purpose, including computational resources and cost considerations. Can other AI techniques be used to reduce or complement the use of LLMs to make the process more efficient?
- Do dedicated sentiment analysis models perform better than LLMs for sentiment analysis in this context? For example, Amazon Comphrehend, Azure Language, or Hugging Face sentiment analysis models?

### What is out of scope?
To keep the experiment focused and manageable, the following aspects are out of scope:
- Production-grade streaming system architecture, scaling, and performance tuning.

### Example System Context
```mermaid
flowchart TD
    DataSources["News Feeds & Data Sources<br/>(e.g., BBC News RSS, Reuters)"]
    FeedListener["Feed Listener System<br/>(Ingests, filters & analyses)"]
    Bedrock["LLM Provider<br/>(Amazon Bedrock / Azure AI Foundry)"]
    Sentiment["Sentiment Analysis<br/>(Amazon Comprehend / Azure Language / Dedicated Models)"]
    Analyst["Analyst<br/>(Consumes insights)"]

    DataSources -->|"Provides articles<br/>(RSS/API)"| FeedListener
    FeedListener <-->|"Assess relevance &<br/>generate embeddings"| Bedrock
    FeedListener <-->|"Analyse sentiment"| Sentiment
    FeedListener -->|"Filtered articles<br/>& insights"| Analyst
```


## Environment Setup
We use `python-dotenv` to manage environment variables for this experiment, which loads variables from `notebooks/.env`. For instructions on setting up your environment along with prerequisites, please refer to the [README.md](../README.md) file in the root of the repository.

In [2]:
import os

import dotenv

dotenv.load_dotenv(dotenv_path=".env")

True

## Source Data
Our source dataset for this experiment is a BBC News RSS feed from the evening of 09 February 2026. This is located at `data/bbc_news_rss_feed_2026-02-09.xml` in this repository. The feed contains the BBC News front page articles, which we will use to test our feed listener system. Each article in the feed includes a title, summary, publication date, and a link to the full article.

> [!IMPORTANT] - As the RSS feed is returned with no filter, it is possible some articles might cover a sensitive topic. We found during the course of this experiment that sometimes these types of articles can trigger LLM-level guardrails, which means the LLM will refuse to process the article and return an error instead. This is something to be aware of when working with real-world data feeds, as it can impact the performance and reliability of your feed listener system.
> For the purposes of this experiment, we have removed any articles that triggered guardrails from the dataset, but in a production system you would need to consider how to handle such cases, whether that's by implementing additional filtering, using a different LLM provider with less restrictive guardrails, or by having a fallback mechanism in place.

In [3]:
import feedparser
import pydantic


class RssFeedEntry(pydantic.BaseModel):
    title: str
    link: str
    published: str
    summary: str


# BBC News Front Page RSS
bbc_rss_path = "./data/2026-02-09-bbc-news-rss.xml"

bbc_feed = feedparser.parse(bbc_rss_path)

articles = [RssFeedEntry(**entry) for entry in bbc_feed.entries]

In [4]:
import openai
import pydantic_ai
from pydantic_ai.embeddings import openai as pydantic_ai_openai_embeddings
from pydantic_ai.models import openai as pydantic_ai_openai_models
from pydantic_ai.providers import azure as pydantic_ai_azure_providers

openai_client = openai.AsyncOpenAI(
    base_url=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

gpt5_mini = pydantic_ai_openai_models.OpenAIChatModel(
    model_name="gpt-5-mini",
    provider=pydantic_ai_azure_providers.AzureProvider(openai_client=openai_client),
)

ada_embeddings = pydantic_ai_openai_embeddings.OpenAIEmbeddingModel(
    model_name="text-embedding-ada-002",
    provider=pydantic_ai_azure_providers.AzureProvider(openai_client=openai_client),
)

## First Pass - LLM Relevance Filtering
For the first pass we use an LLM to judge each article’s relevance from its title and summary. The model is prompted with a concise set of keywords, topics, or targeted questions about the organisation or department, and asked to return a binary Yes/No indicating whether the article is relevant.

> [!IMPORTANT]
> we avoid numeric relevance scores (e.g., 1–10) because LLMs can produce inconsistent or hallucinatory scales; requesting a binary classification reduces that risk and makes filtering straightforward.
> 
> If finer granularity is needed, a dedicated classifier or a hybrid approach (e.g., embeddings + similarity scoring or a supervised ranking model) will likely be more suitable and reliable than asking an LLM for a relevance score.

In [5]:
import dataclasses


@dataclasses.dataclass
class Deps:
    keywords: list[str]
    topics: list[str]
    questions: list[str]


agent = pydantic_ai.Agent(deps_type=Deps, output_type=bool)


@agent.system_prompt
def system_prompt(ctx: pydantic_ai.RunContext[Deps]) -> str:
    return f"""
    You are an intelligent assistant designed to filter news articles for relevance to a specific organization or department.
    Your task is to determine whether each article is relevant based on its title and summary, in relation to the following context:

    Keywords: {", ".join(ctx.deps.keywords)}
    Topics: {", ".join(ctx.deps.topics)}
    Questions: {", ".join(ctx.deps.questions)}

    For each article, you will receive its title and summary.
    You should return "Yes" if the article is relevant to the organization/department based on the provided context, and "No" if it is not.
    """

In [6]:
async def llm_check_articles(
    articles: list[RssFeedEntry],
    agent: pydantic_ai.Agent[Deps, bool],
    deps: Deps,
    model: pydantic_ai_openai_models.BaseModel,
) -> tuple[list[RssFeedEntry], list[RssFeedEntry]]:
    """Filter articles by relevance using sequential LLM calls to respect rate limits.

    Returns:
        tuple of (relevant_articles, not_relevant_articles) for easy comparison
    """
    relevant_articles = []
    not_relevant_articles = []

    for article in articles:
        print("Checking article:", article.title)

        try:
            result = await agent.run(
                deps=deps,
                model=model,
                user_prompt=f"Title: {article.title}\nSummary: {article.summary}\nIs this article relevant?",
            )

            if result.output:
                relevant_articles.append(article)
            else:
                not_relevant_articles.append(article)
        except Exception as e:
            print(f"Error checking article '{article.title}': {e}")

    return relevant_articles, not_relevant_articles

In [7]:
defra_deps = Deps(
    keywords=["uk government", "parliament"],
    topics=["politics", "government policy", "flooding"],
    questions=[
        "Is the article about the UK government or its policies?",
        "Does the article discuss political events or decisions that could impact Defra?",
        "Is the article relevant to current political issues in the UK?",
    ],
)

In [8]:
gpt5_relevant, gpt5_not_relevant = await llm_check_articles(
    articles, agent, defra_deps, gpt5_mini
)

Checking article: Cabinet ministers rally round PM as Sarwar calls for him to quit
Checking article: Scottish Labour leader makes his biggest political gamble - but can it pay off?
Checking article: 'No regrets' - Vonn sustains 'complex tibia fracture'
Checking article: Savannah Guthrie pleads for help as missing mother's ransom deadline looms
Checking article: BBC assesses weaponry used to massacre Iran's protesters
Checking article: Streeting's Mandelson messages reveal election fears and criticism of government
Checking article: Palestinians say new Israeli measures in West Bank amount to de facto annexation
Checking article: £5bn council SEND debts to be paid off by government
Checking article: Family of Hillsborough victims criticise 'hollow' police apology
Checking article: Airport safety concerns ground flights from remote British overseas island of St Helena
Checking article: So close to a 'world first' - fourth for Brookes on frustrating day for Team GB
Checking article: Olymp

## First Pass Results
As expected, the LLM was more than capable of filtering articles for relevance according to the criteria we set. However, we found that it was not very efficient at this task, as it required a call to the LLM for each article in the feed, which quickly adds up in terms of cost and latency. Most of the articles were filtered out as not relevant, which is good in terms of accuracy, but it means we made a lot of LLM calls to achieve that result.

### Side Note - Structured Output with Pydantic AI
As an interesting side note - During this experiment we found out how Pydantic AI implements LLM's outputting to it's output type. As a reminder, one of the main selling points of the framework is that it can output structured data types according to a Pydantic schema or basic types. In this case, we wanted the LLM to output a boolean value indicating relevance, so we just specified the output type as `bool` in the `Agent` definition.

We found that when using GPT-5, the model consistently returned a boolean value as expected, which made it very easy to filter the articles based on relevance and avoid any additional parsing or error handling. However, when we switched to Phi-4, we found that the model API returned an error with our request due to `ToolConfig`. After some investigation, we found that this was because Phi-4 does not support tool calling at all. This led us down the path of discovering Pydantic AI registers a tool that handles enforcing the output type(s).

## Second Pass - Embedding-Based Relevance Filtering

Instead of using an LLM to assess relevance, we can use embeddings to calculate semantic similarity between articles and our relevance criteria. This approach:
- Is more efficient (no LLM inference needed for filtering)
- Is more cost-effective (embeddings are cheaper than LLM calls)
- Provides numeric similarity scores for fine-grained filtering
- Is more sustainable for high-volume feeds

We'll use cosine similarity to compare article embeddings against embeddings of our keywords, topics, and questions.

In [9]:
from dataclasses import dataclass

import numpy as np
from pydantic_ai.embeddings import Embedder


def cosine_similarity(vec1: list[float], vec2: list[float]) -> float:
    """Calculate cosine similarity between two vectors."""
    a = np.array(vec1)
    b = np.array(vec2)
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


@dataclass
class ArticleRelevanceScore:
    """Detailed relevance breakdown for an article."""

    article: RssFeedEntry
    keyword_scores: dict[str, float]
    topic_scores: dict[str, float]
    question_scores: dict[str, float]
    avg_keyword_score: float
    avg_topic_score: float
    avg_question_score: float

    def __repr__(self):
        return f"ArticleRelevanceScore(title='{self.article.title[:30]}...', keywords={self.avg_keyword_score:.3f}, topics={self.avg_topic_score:.3f}, questions={self.avg_question_score:.3f})"


async def embedding_check_articles(
    articles: list[RssFeedEntry],
    deps: Deps,
    embedding_model: pydantic_ai_openai_embeddings.OpenAIEmbeddingModel,
    relevance_threshold: float = 0.75,
    weights: dict[str, float] | None = None,
) -> tuple[list[ArticleRelevanceScore], list[ArticleRelevanceScore]]:
    """Filter articles by relevance using embedding similarity with granular scoring.

    This approach compares each article against keywords, topics, and questions individually,
    providing detailed insight into what makes an article relevant.

    Args:
        articles: List of RSS feed entries to check
        deps: Dependencies containing keywords, topics, and questions
        embedding_model: The embedding model to use
        relevance_threshold: Minimum overall score to consider relevant (0-1)
        weights: Optional weights for combining scores. Defaults to equal weights.
                Keys: 'keywords', 'topics', 'questions'

    Returns:
        tuple of (relevant_articles_with_scores, not_relevant_articles_with_scores)
    """
    if weights is None:
        weights = {"keywords": 0.25, "topics": 0.25, "questions": 0.5}

    embedder = Embedder(model=embedding_model)

    print("Generating context embeddings...")

    keyword_embeddings = {}
    if deps.keywords:
        keyword_results = await embedder.embed_query(deps.keywords)
        keyword_embeddings = dict(
            zip(deps.keywords, keyword_results.embeddings, strict=True)
        )
        print(f"  Embedded {len(deps.keywords)} keywords")

    topic_embeddings = {}
    if deps.topics:
        topic_results = await embedder.embed_query(deps.topics)
        topic_embeddings = dict(zip(deps.topics, topic_results.embeddings, strict=True))
        print(f"  Embedded {len(deps.topics)} topics")

    question_embeddings = {}
    if deps.questions:
        question_results = await embedder.embed_query(deps.questions)
        question_embeddings = dict(
            zip(deps.questions, question_results.embeddings, strict=True)
        )
        print(f"  Embedded {len(deps.questions)} questions")

    relevant_articles = []
    not_relevant_articles = []

    for article in articles:
        article_text = f"{article.title} {article.summary}"
        print(f"\nChecking article: {article.title[:50]}...")

        try:
            article_result = await embedder.embed_documents(article_text)
            article_embedding = article_result.embeddings[0]

            keyword_scores = {
                kw: cosine_similarity(emb, article_embedding)
                for kw, emb in keyword_embeddings.items()
            }

            topic_scores = {
                topic: cosine_similarity(emb, article_embedding)
                for topic, emb in topic_embeddings.items()
            }

            question_scores = {
                q: cosine_similarity(emb, article_embedding)
                for q, emb in question_embeddings.items()
            }

            avg_keyword_score = (
                sum(keyword_scores.values()) / len(keyword_scores)
                if keyword_scores
                else 0.0
            )

            avg_topic_score = (
                sum(topic_scores.values()) / len(topic_scores) if topic_scores else 0.0
            )
            
            avg_question_score = (
                sum(question_scores.values()) / len(question_scores)
                if question_scores
                else 0.0
            )

            relevant = np.average(
                [avg_keyword_score, avg_topic_score, avg_question_score],
                weights=[weights["keywords"], weights["topics"], weights["questions"]],
            )

            score = ArticleRelevanceScore(
                article=article,
                keyword_scores=keyword_scores,
                topic_scores=topic_scores,
                question_scores=question_scores,
                avg_keyword_score=avg_keyword_score,
                avg_topic_score=avg_topic_score,
                avg_question_score=avg_question_score,
            )

            print(score)

            if relevant >= relevance_threshold:
                relevant_articles.append(article)
            else:
                not_relevant_articles.append(article)

        except Exception as e:
            print(f"Error embedding article '{article.title}': {e}")

    return relevant_articles, not_relevant_articles

In [10]:
# Run embedding-based relevance check with the same DEFRA context
# You can adjust weights to emphasize different components
# e.g., weights={'keywords': 0.5, 'topics': 0.3, 'questions': 0.2}
embedding_relevant, embedding_not_relevant = await embedding_check_articles(
    articles,
    defra_deps,
    ada_embeddings,
    relevance_threshold=0.75,  # Adjust this threshold based on results
)

print(f"Relevant articles: {len(embedding_relevant)}")
print(f"Not relevant articles: {len(embedding_not_relevant)}")

Generating context embeddings...
  Embedded 2 keywords
  Embedded 3 topics
  Embedded 3 questions

Checking article: Cabinet ministers rally round PM as Sarwar calls f...
ArticleRelevanceScore(title='Cabinet ministers rally round ...', keywords=0.785, topics=0.727, questions=0.761)

Checking article: Scottish Labour leader makes his biggest political...
ArticleRelevanceScore(title='Scottish Labour leader makes h...', keywords=0.754, topics=0.721, questions=0.772)

Checking article: 'No regrets' - Vonn sustains 'complex tibia fractu...
ArticleRelevanceScore(title=''No regrets' - Vonn sustains '...', keywords=0.721, topics=0.723, questions=0.712)

Checking article: Savannah Guthrie pleads for help as missing mother...
ArticleRelevanceScore(title='Savannah Guthrie pleads for he...', keywords=0.732, topics=0.734, questions=0.728)

Checking article: BBC assesses weaponry used to massacre Iran's prot...
ArticleRelevanceScore(title='BBC assesses weaponry used to ...', keywords=0.761, topics=0

## Second Pass Results
The embedding-based filtering was able to identify a similar set of relevant articles as the LLM, but with significantly fewer computational resources and at a much lower cost. By setting an appropriate relevance threshold, we were able to filter out a large portion of the irrelevant articles while retaining most of the relevant ones.

However, we did find that the embedding-based approach was not perfect although it mostly (apart from one) captured the same relevant articles along with a few additional ones that the LLM said were not relevant. This could be due to that embeddings capture semantic similarity but do not have the same level of contextual understanding as an LLM, which can lead to some false positives or false negatives depending on the specific content of the articles and how well it matches the relevance criteria.

In [51]:
import pandas as pd

# Compare LLM Agent vs Embedding Model
gpt5_total_relevant = len(gpt5_relevant)
gpt5_total_not_relevant = len(gpt5_not_relevant)
embedding_total_relevant = len(embedding_relevant)
embedding_total_not_relevant = len(embedding_not_relevant)

# Create sets of titles for comparison
gpt5_relevant_titles = {article.title for article in gpt5_relevant}
embedding_relevant_titles = {article.title for article in embedding_relevant}

# Find differences
both_relevant = gpt5_relevant_titles & embedding_relevant_titles
only_gpt5 = gpt5_relevant_titles - embedding_relevant_titles
only_embedding = embedding_relevant_titles - gpt5_relevant_titles

# Create summary table
summary_data = {
    "Metric": [
        "Relevant Articles",
        "Not Relevant Articles",
        "Total Articles",
        "",
        "Agreed as Relevant (Both)",
        "Only LLM Found Relevant",
        "Only Embedding Found Relevant",
    ],
    "LLM Agent (GPT-5)": [
        gpt5_total_relevant,
        gpt5_total_not_relevant,
        gpt5_total_relevant + gpt5_total_not_relevant,
        "",
        len(both_relevant),
        len(only_gpt5),
        "-",
    ],
    "Embedding Model": [
        embedding_total_relevant,
        embedding_total_not_relevant,
        embedding_total_relevant + embedding_total_not_relevant,
        "",
        len(both_relevant),
        "-",
        len(only_embedding),
    ],
    "Difference": [
        embedding_total_relevant - gpt5_total_relevant,
        embedding_total_not_relevant - gpt5_total_not_relevant,
        0,
        "",
        "",
        "",
        "",
    ],
}

summary_df = pd.DataFrame(summary_data)

summary_df

,Metric,LLM Agent (GPT-5),Embedding Model,Difference
0,Relevant Articles,4,8,4
1,Not Relevant Articles,28,24,-4
2,Total Articles,32,32,0
3,,,,
4,Agreed as Relevant (Both),3,3,
5,Only LLM Found Relevant,1,-,
6,Only Embedding Found Relevant,-,5,


## Third Pass - Embedding into LLM Agent (Hybrid Approach)

This hybrid approach combines the best of both methods:
1. **Fast Pre-filtering with Embeddings**: Use semantic similarity to quickly filter out clearly irrelevant articles (cheap, fast, sustainable)
2. **LLM Analysis on Filtered Subset**: Apply the LLM agent only to pre-filtered articles for contextual understanding and nuanced judgment

Benefits:
- **Cost-effective**: Reduces LLM calls by ~50-80% depending on the embedding threshold
- **Sustainable**: Lower computational overhead and token usage
- **High Quality**: LLM still provides contextual analysis where it matters most
- **Scalable**: Can handle high-volume feeds efficiently

In [11]:
# Apply LLM agent only to embedding-filtered articles
# This reuses the embedding_relevant list from the second pass
hybrid_relevant, hybrid_not_relevant = await llm_check_articles(
    embedding_relevant,  # Only check pre-filtered articles
    agent,
    defra_deps,
    gpt5_mini,
)

print(
    f"Embedding pre-filter: {len(embedding_relevant)}/{len(articles)} articles"
)
print(f"LLM final filter: {len(hybrid_relevant)}/{len(embedding_relevant)} articles")
print(
    f"Total LLM calls saved: {len(articles) - len(embedding_relevant)} ({(1 - len(embedding_relevant) / len(articles)) * 100:.1f}%)"
)

Checking article: Cabinet ministers rally round PM as Sarwar calls for him to quit
Checking article: Scottish Labour leader makes his biggest political gamble - but can it pay off?
Checking article: BBC assesses weaponry used to massacre Iran's protesters
Checking article: Streeting's Mandelson messages reveal election fears and criticism of government
Checking article: £5bn council SEND debts to be paid off by government
Checking article: Businesses face extinction unless they protect nature, major report warns
Checking article: Iran arrests reformists as crackdown on dissent widens, reports say
Checking article: BBC News app
Embedding pre-filter: 8/32 articles
LLM final filter: 4/8 articles
Total LLM calls saved: 24 (75.0%)


In [ ]:
# Compare all three approaches
hybrid_total_relevant = len(hybrid_relevant)
hybrid_total_not_relevant = len(hybrid_not_relevant) + len(
    embedding_not_relevant
)  # Articles rejected by embedding + LLM

# Create sets for comparison
hybrid_relevant_titles = {article.title for article in hybrid_relevant}

# Find agreement patterns
all_three_agree = (
    gpt5_relevant_titles & embedding_relevant_titles & hybrid_relevant_titles
)
gpt5_hybrid_agree = gpt5_relevant_titles & hybrid_relevant_titles
embedding_hybrid_agree = embedding_relevant_titles & hybrid_relevant_titles

# Create comprehensive comparison table
comparison_data = {
    "Metric": [
        "Relevant Articles",
        "Not Relevant Articles",
        "Total Articles",
        "LLM Calls Made",
        "",
        "All 3 Methods Agree (Relevant)",
        "LLM-Only & Hybrid Agree",
        "Embedding-Only & Hybrid Agree",
        "",
        "Efficiency (LLM Calls Saved)",
        "Percentage of LLM Calls Saved",
    ],
    "LLM Only": [
        gpt5_total_relevant,
        gpt5_total_not_relevant,
        len(articles),
        len(articles),
        "",
        len(all_three_agree),
        len(gpt5_hybrid_agree),
        "-",
        "",
        0,
        "0%",
    ],
    "Embedding Only": [
        embedding_total_relevant,
        embedding_total_not_relevant,
        len(articles),
        0,
        "",
        len(all_three_agree),
        "-",
        len(embedding_hybrid_agree),
        "",
        len(articles),
        "100%",
    ],
    "Hybrid (Embedding→LLM)": [
        hybrid_total_relevant,
        hybrid_total_not_relevant,
        len(articles),
        len(embedding_relevant),
        "",
        len(all_three_agree),
        len(gpt5_hybrid_agree),
        len(embedding_hybrid_agree),
        "",
        len(articles) - len(embedding_relevant),
        f"{(1 - len(embedding_relevant) / len(articles)) * 100:.1f}%",
    ],
}

comparison_df = pd.DataFrame(comparison_data)
comparison_df

,Metric,LLM Only,Embedding Only,Hybrid (Embedding→LLM)
0,Relevant Articles,4,8,4
1,Not Relevant Articles,28,24,28
2,Total Articles,32,32,32
3,LLM Calls Made,32,0,8
4,,,,
5,All 3 Methods Agree (Relevant),3,3,3
6,LLM-Only & Hybrid Agree,3,-,3
7,Embedding-Only & Hybrid Agree,-,4,4
8,,,,
9,Efficiency (LLM Calls Saved),0,32,24


In [ ]:
# Analyze differences between approaches
only_llm = gpt5_relevant_titles - hybrid_relevant_titles
only_hybrid = hybrid_relevant_titles - gpt5_relevant_titles

if only_llm:
    print(f"LLM-Only found relevant but Hybrid rejected ({len(only_llm)}):")
    for title in only_llm:
        print(f"  - {title}")
    print()

if only_hybrid:
    print(f"Hybrid found relevant but LLM-Only rejected ({len(only_hybrid)}):")
    for title in only_hybrid:
        print(f"  - {title}")
    print()

if all_three_agree:
    print(f"All 3 methods agree these are relevant ({len(all_three_agree)}):")
    for title in all_three_agree:
        print(f"  - {title}")

=== Articles where methods disagree ===

LLM-Only found relevant but Hybrid rejected (1):
  - More than 90 flood warnings in place across UK as forecasters warn 'no sign' of dry spell

Hybrid found relevant but LLM-Only rejected (1):
  - Scottish Labour leader makes his biggest political gamble - but can it pay off?

All 3 methods agree these are relevant (3):
  - £5bn council SEND debts to be paid off by government
  - Streeting's Mandelson messages reveal election fears and criticism of government
  - Cabinet ministers rally round PM as Sarwar calls for him to quit
